In [1]:
!git clone https://github.com/BarathSrinivasan2002/Deep_Learning_Face_mask_detection.git

Cloning into 'Deep_Learning_Face_mask_detection'...
remote: Enumerating objects: 1721, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 1721 (delta 2), reused 6 (delta 2), pack-reused 1715 (from 2)
Receiving objects: 100% (1721/1721), 395.24 MiB | 16.44 MiB/s, done.
Resolving deltas: 100% (837/837), done.
Updating files: 100% (1711/1711), done.


In [2]:
%cd Deep_Learning_Face_mask_detection

/content/Deep_Learning_Face_mask_detection


In [3]:
!ls

face-mask-detection  labels.csv  README.md  requirements.txt  Supervised_Learning.ipynb


In [4]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import cv2
import os
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt

In [5]:
# Function to parse XML annotations
def parse_xml(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    annotations = []

    for obj in root.findall('object'):
        filename = root.find('filename').text
        label = obj.find('name').text
        bbox = obj.find('bndbox')
        coords = {
            'xmin': int(bbox.find('xmin').text),
            'ymin': int(bbox.find('ymin').text),
            'xmax': int(bbox.find('xmax').text),
            'ymax': int(bbox.find('ymax').text)
        }
        annotations.append({'filename': filename, 'label': label, 'bbox': coords})

    return annotations

In [7]:
# Paths to dataset
ANNOTATION_PATH = '/content/Deep_Learning_Face_mask_detection/face-mask-detection/versions/1/annotations'
IMAGE_PATH = '/content/Deep_Learning_Face_mask_detection/face-mask-detection/versions/1/images'

# Load XML files
xml_files = [os.path.join(ANNOTATION_PATH, f) for f in os.listdir(ANNOTATION_PATH) if f.endswith('.xml')]
data = []

In [8]:
# Extract annotations
for xml_file in xml_files:
    annotations = parse_xml(xml_file)
    for annotation in annotations:
        data.append([annotation['filename'], annotation['label'], annotation['bbox']])

# Create DataFrame
df = pd.DataFrame(data, columns=['filename', 'label', 'bbox'])
print(df['label'].value_counts())

label
with_mask                3232
without_mask              717
mask_weared_incorrect     123
Name: count, dtype: int64


In [9]:
# Label mapping
label_map = {'with_mask': 0, 'without_mask': 1, 'mask_weared_incorrect': 2}

In [11]:
# Image processing
face_images = []
face_labels = []

for _, row in df.iterrows():
    bbox = row['bbox']
    image = cv2.imread(os.path.join(IMAGE_PATH, row['filename']))
    image = image[bbox['ymin']:bbox['ymax'], bbox['xmin']:bbox['xmax']]
    image = cv2.resize(image, (224, 224))
    image = img_to_array(image)
    image = preprocess_input(image)  # Normalize using MobileNetV2 preprocessing

    face_images.append(image)
    face_labels.append(label_map[row['label']])

face_images = np.array(face_images, dtype='float32')
face_labels = to_categorical(np.array(face_labels))  # Convert labels to categorical format


In [12]:
# Data Augmentation
augmentation = ImageDataGenerator(
    zoom_range=0.1,
    shear_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    rotation_range=25,
    fill_mode='nearest'
)

# Splitting Data
train_x, test_val_x, train_y, test_val_y = train_test_split(face_images, face_labels, test_size=0.3, stratify=face_labels)
test_x, val_x, test_y, val_y = train_test_split(test_val_x, test_val_y, test_size=0.5, stratify=test_val_y)


In [13]:
print(f"Train: {train_x.shape}, Validation: {val_x.shape}, Test: {test_x.shape}")

Train: (2850, 224, 224, 3), Validation: (611, 224, 224, 3), Test: (611, 224, 224, 3)


In [14]:
### Model 1: MobileNetV2
mobilenet = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
mobilenet.trainable = False  # Freeze base model

model_mobilenet = Sequential([
    mobilenet,
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

model_mobilenet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(model_mobilenet.summary())

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224 (Functional)    │ (None, 7, 7, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 62720)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       8,028,288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 3)                   │             387 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 10,286,659 (39.24 MB)

 Trainable params: 8,028,675 (30.63 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

None


In [15]:
# Training MobileNet
history_mobilenet = model_mobilenet.fit(
    augmentation.flow(train_x, train_y, batch_size=32),
    validation_data=(val_x, val_y),
    epochs=10
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 55s 486ms/step - accuracy: 0.7583 - loss: 5.9893 - val_accuracy: 0.9002 - val_loss: 0.3408
Epoch 2/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 30s 337ms/step - accuracy: 0.8528 - loss: 0.4967 - val_accuracy: 0.8985 - val_loss: 0.2948
Epoch 3/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 33s 362ms/step - accuracy: 0.8590 - loss: 0.4559 - val_accuracy: 0.9264 - val_loss: 0.2817
Epoch 4/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 32s 359ms/step - accuracy: 0.8566 - loss: 0.4428 - val_accuracy: 0.9198 - val_loss: 0.2716
Epoch 5/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 30s 337ms/step - accuracy: 0.8841 - loss: 0.3511 - val_accuracy: 0.9034 - val_loss: 0.3068
Epoch 6/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 32s 351ms/step - accuracy: 0.8757 - loss: 0.3407 - val_accuracy: 0.9198 - val_loss: 0.2435
Epoch 7/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 30s 338ms/step - accuracy: 0.8798 - loss: 0.3195 - val_accuracy: 0.9247 - val_loss: 0.2385
Epoch 8/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 32s 357ms/step - accuracy: 0.8919 - loss: 0.2704 - val_accu

In [16]:
### Model 2: Basic CNN
model_cnn = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

model_cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(model_cnn.summary())

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 186624)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │      23,888,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 3)                   │             387 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 23,907,779 (91.20 MB)

 Trainable params: 23,907,779 (91.20 MB)

 Non-trainable params: 0 (0.00 B)

None


In [17]:
# Training Basic CNN
history_cnn = model_cnn.fit(
    augmentation.flow(train_x, train_y, batch_size=32),
    validation_data=(val_x, val_y),
    epochs=10
)

Epoch 1/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 41s 397ms/step - accuracy: 0.8252 - loss: 1.5736 - val_accuracy: 0.9329 - val_loss: 0.2556
Epoch 2/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 32s 353ms/step - accuracy: 0.9182 - loss: 0.2593 - val_accuracy: 0.9362 - val_loss: 0.2364
Epoch 3/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 30s 338ms/step - accuracy: 0.9299 - loss: 0.2505 - val_accuracy: 0.9345 - val_loss: 0.2423
Epoch 4/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 33s 365ms/step - accuracy: 0.9332 - loss: 0.2518 - val_accuracy: 0.9345 - val_loss: 0.2368
Epoch 5/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 30s 338ms/step - accuracy: 0.9307 - loss: 0.2339 - val_accuracy: 0.9329 - val_loss: 0.2185
Epoch 6/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 32s 351ms/step - accuracy: 0.9318 - loss: 0.2304 - val_accuracy: 0.9313 - val_loss: 0.2505
Epoch 7/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 30s 338ms/step - accuracy: 0.9344 - loss: 0.2214 - val_accuracy: 0.9345 - val_loss: 0.2698
Epoch 8/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 31s 345ms/step - accuracy: 0.9370 - loss: 0.2199 - val_accu

In [18]:
# Evaluating Models
mobilenet_eval = model_mobilenet.evaluate(test_x, test_y)
cnn_eval = model_cnn.evaluate(test_x, test_y)

print(f"MobileNetV2 Test Accuracy: {mobilenet_eval[1]:.4f}")
print(f"Basic CNN Test Accuracy: {cnn_eval[1]:.4f}")

20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.9252 - loss: 0.2220
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9457 - loss: 0.1753
MobileNetV2 Test Accuracy: 0.9247
Basic CNN Test Accuracy: 0.9444
